In [1]:
import numpy as np
import csv
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import pickle
from glob import glob
import seaborn as sns
import matplotlib.pylab as plt
torch.set_default_dtype(torch.float32)
"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True
1
GeForce GTX 1080 Ti


In [2]:
ROOT_PATH = os.getcwd() + "/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    f_in = ROOT_PATH + "train" + "/" + city + "_inputs"
    #print(f_in)
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    
    outputs = None
    
    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)
        
    if split=="val_train":
        f_out = ROOT_PATH + "train" + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        
        length = len(inputs)
        split_size = int(length * 0.8)
        outputs = np.asarray(outputs)[:split_size]
        inputs = inputs[:split_size]
        
    if split=="val":
        f_out = ROOT_PATH + "train" + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        
        length = len(inputs)
        split_size = int(length * 0.8)
        outputs = np.asarray(outputs)[split_size:]
        inputs = inputs[split_size:]
        
    return inputs, outputs



In [3]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.city = city
        self.split = split
        self.transform = transform

        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=True)
        self.global_mean = np.mean(self.inputs, axis = (0,1), keepdims = True)
        self.global_std = np.std(np.sqrt(self.inputs[:, :, 0]**2 + self.inputs[:, :, 0]**2))
        self.inputs = (self.inputs - self.global_mean)/self.global_std
        
        if split != 'test' or split != 'val':
            self.outputs = (self.outputs - self.global_mean)/self.global_std    
    
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        
        if self.split == "train" or self.split == "val_train" or self.split == "val":
            data = (self.inputs[idx], self.outputs[idx])
        else:
            data = self.inputs[idx]
            
        if self.transform:
            data = self.transform(data)

        return data

In [4]:
class LinearRegression(torch.nn.Module):
    def __init__(self, input_dim, out_dim):
        super(LR, self).__init__()
        """
        the __init__() method that defines the layers and other components
        """ 
        self.model = nn.Linear(input_dim, out_dim) # input_dim = input_length*2
        
    def forward(self, x, output_steps): 
        """
        the forward function is where computatioin gets done
        """
        x = x.reshape(x.shape[0], -1)
        
        outputs = []
        for i in range(output_steps):
            out = self.model(x)    
            outputs.append(out)
            x = torch.cat([x[:,2:],  out], dim = 1)
            
        outputs = torch.cat(outputs, dim = 1)
        return outputs.reshape(outputs.shape[0], output_steps, 2)
    
model = LinearRegression(input_dim = 50 * 2, out_dim = 2)

NameError: name 'LR' is not defined

In [5]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout_rate):
        
        super(Encoder, self).__init__()

        self.lstm = nn.LSTM(input_size = input_dim, 
                            hidden_size = hidden_dim, 
                            num_layers= num_layers, 
                            dropout = dropout_rate, 
                            batch_first = True)
        
        
    def forward(self, source):
        
        # hidden = (h, c)
        # h, c: num_layers x bz x  hid_dim
        # outputs: bz x input_length x hid_dim
        outputs, hidden = self.lstm(source)
        
        return outputs, hidden
    
class AttnDecoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers, dropout_rate):

        super(AttnDecoder, self).__init__()

        # Learn the attention scores
        self.attn = nn.Linear(hidden_dim + output_dim, 50)
        
        # Learn the final input to the decoder 
        self.attn_combine = nn.Linear(hidden_dim + output_dim, hidden_dim)
        
        # Decoder LSTM
        self.lstm = nn.LSTM(input_size = hidden_dim, 
                            hidden_size = hidden_dim, 
                            num_layers= num_layers, 
                            dropout = dropout_rate, 
                            batch_first = True)
        
        self.output_layer = nn.Linear(hidden_dim, output_dim)
      
    def forward(self, x, hidden, encoder_outputs):
        
        h = hidden[0]
        h = h.transpose(0,1).reshape(h.shape[1], -1)
        
        # Compute Attention Scores
        attn_weights = F.softmax(self.attn(torch.cat([x, h], 1)), dim =1)
        
        # Calculate weighted sum of encoder hidden states     
        attn_applied = torch.einsum("bl,blh->bh", attn_weights, encoder_outputs)
        
        x = torch.cat((x, attn_applied), dim = 1)
        x = self.attn_combine(x).unsqueeze(1)
        x = F.relu(x)
        
        output, decoder_hidden= self.lstm(x, hidden)  
        prediction = self.output_layer(output.float())
        
        return prediction.squeeze(1), decoder_hidden
    
class Seq2Seq(nn.Module):
    def __init__(self, input_dim = 2, output_dim = 2, hidden_dim = 128, num_layers = 1, dropout_rate = 0.3):
        
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, num_layers, dropout_rate)
        self.decoder = AttnDecoder(output_dim, hidden_dim, num_layers, dropout_rate)

    def forward(self, source, target_length):

        batch_size = source.size(0) 
        input_length = source.size(1) 
        
        encoder_outputs, concat_hidden = self.encoder(source)
        
        # the last encoder hidden state is used as initial hidden state of the decoder
        decoder_hidden = concat_hidden
        # the first input to the decoder is last input position
        decoder_output = source[:,-1]
    
        
        outputs = torch.zeros(batch_size, target_length, 2)
        for t in range(target_length):    
            decoder_output, decoder_hidden = self.decoder(decoder_output, decoder_hidden, encoder_outputs)
            outputs[:,t] = decoder_output   
        return outputs

In [6]:
batch_sz = 128  # batch size 

In [7]:
class EncoderModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32)
        ).to(device)
        
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 120),
            nn.ReLU(),
            nn.Linear(120, 120)
        ).to(device)
        
    def forward(self, x):
        x = x.reshape(-1, 100).float()
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.reshape(-1, 60, 2)
        x = x.to(device)
        return x

In [9]:
df = pd.read_csv('sample_submission.csv', index_col='ID')
criterion = nn.MSELoss().to(device)

def run_model(city):
    
    model = EncoderModel()
    model = model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    train_dataset = ArgoverseDataset(city = city, split = "train")
    train_loader = DataLoader(train_dataset, batch_size=batch_sz)
    global_std = train_dataset.global_std
    global_mean = train_dataset.global_mean
    
    for epoch in range(10):
        total_loss = 0
        for i_batch, sample_batch in enumerate(train_loader):
            inp, out = sample_batch
            inp = inp.float().to(device)
            out = out.float().to(device)
            preds = model(inp)
            preds = preds.float().to(device)
            loss = criterion(preds, out)
#             inversedPreds = preds.detach().numpy() * global_std * global_mean

            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
        print('city: {} epoch {}: loss = {}'.format(city, epoch, total_loss))
#     predict(city, model)
    validate(model, city)
    
    
def predict(city, model):
    
    test_dataset  = ArgoverseDataset(city = city, split = 'test')
    test_loader = DataLoader(test_dataset, batch_size=batch_sz)
    
    global_std = test_dataset.global_std
    global_mean = test_dataset.global_mean
    
    for i_batch, sample_batch in enumerate(test_loader):
        inp = sample_batch
        inp = inp.float().to(device)
        preds = model(inp)
        preds = preds.detach().numpy() * global_std + global_mean
        for i in range(len(preds[0])):
            for j in range(len(preds)):
                row = str(i_batch * batch_sz + j) + "_" + city
                x_col = "v" + str(2 * i)
                y_col = "v" + str(2 * i + 1)
                df.loc[row, x_col] = preds[j][i][0].item()
                df.loc[row, y_col] = preds[j][i][1].item()
        
    df.to_csv('sample_submission.csv')
    
def validate(model, city):
    
    total_loss = 0
    val_dataset = ArgoverseDataset(city = city, split = "val")
    val_loader = DataLoader(val_dataset, batch_size=batch_sz)
    global_std = val_dataset.global_std
    global_mean = val_dataset.global_mean
    
    for i_batch, sample_batch in enumerate(val_loader):
        inp, out = sample_batch
        inp = inp.float().to(device)
        preds = model(inp)
        preds = preds.float().to(device)
        loss = criterion(preds, out.to(device))
        total_loss += loss.item()
    print('city: {} loss = {}'.format(city, total_loss / len(val_dataset)))
            
for city in cities:
    run_model(city)
# run_model("palo-alto")

city: austin epoch 0: loss = 94.86022264137864
city: austin epoch 1: loss = 4.767970793880522
city: austin epoch 2: loss = 0.4295295405900106
city: austin epoch 3: loss = 0.2657199546811171
city: austin epoch 4: loss = 0.20560628804378211
city: austin epoch 5: loss = 0.17916903569130227
city: austin epoch 6: loss = 0.16435696854023263
city: austin epoch 7: loss = 0.15508097945712507
city: austin epoch 8: loss = 0.14913996600080281
city: austin epoch 9: loss = 0.145627606427297
city: austin loss = 3.4420418736872143e-06
city: miami epoch 0: loss = 84.21882901899517
city: miami epoch 1: loss = 1.9461548190156464
city: miami epoch 2: loss = 0.10735352434130618
city: miami epoch 3: loss = 0.04538698252508766
city: miami epoch 4: loss = 0.03163025977482903
city: miami epoch 5: loss = 0.027421483799116686
city: miami epoch 6: loss = 0.025405450054677203
city: miami epoch 7: loss = 0.0242761859226448
city: miami epoch 8: loss = 0.023760712010698626
city: miami epoch 9: loss = 0.02328895626123